In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

def load_template():
    """
    load correlation image saved in data folder
    """
    with open('./data/corr_image.pkl', 'rb') as f:
        template = joblib.load(f)['template']
    return template

# Normalizing outputs of CNMFE
You have probably noticed that when you run CNMFE on your 1p data, and try to calculate ∆F/F -- `cnm.estimates.F_dff`, which is generated via `cnm.estimates.detrend_df_f()`-- you will get the following warning:

    Results should not be interpreted as DF/F normalized but only as detrended.

In other words, while the algorithm is *detrending* the traces, it is not actually generating a *normalized* ∆F/F trace for the components. 

Why is that? It's because the algorithm that caiman uses to normalize the traces with 2p recordings counts on having a well-behaved baseline. This doesn't work for 1p data because the baseline is dominated by out of focus background fluorescence, which makes the problem ill defined (note: I am not 100% sure why it can't be attacked using some iteration of the ring model used in CNMFE, but let's leave that aside for now as caiman doesn't do that).

The upshot of this is that caiman doesn't generate a normalized signal for your 1p data, but a signal in arbitrary units that makes it hard to compare magnitudes across components. Luckily, there are other powerful techniques you can use to generate a normalization term for your 1p data that will allow for such comparisons. 

This notebook shows one such technique that takes advantage of the spectral noise estimation function from caiman `GetSn()`. The technique was suggested originally by Eftychiios Pnevmatikakis, and is adapted from code written by Zach Berry.

After setting up the imports and building the cnm object (based on the data used in the caiman demo), we will walk through the concepts involved, and then demo some code that shows how to use this technique.

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch
import joblib
import bokeh.plotting as bpl
import holoviews as hv

import caiman as cm
from caiman.source_extraction.cnmf.cnmf import load_CNMF
from caiman.source_extraction import cnmf
from caiman.source_extraction.cnmf.deconvolution import GetSn

bpl.output_notebook()
hv.notebook_extension('bokeh')
plt.rcParams["font.size"] = "14"
print(f"Using python {sys.version}")
print(f"Holoviews {hv.__version__}\nCaiman {cm.__version__}")

## Build cnmf object
Start cluster and build object from `hdf5` file saved from data already analyzed in the CNMFE demo in caiman:

In [ ]:
cnmfe_results_path = r'./data/cnmfe_demo.hdf5'
num_cpus = 2 # use however many you need: just leave a few so your RAM stays happy
# note if a cluster already exists it will be closed so a new session will be opened
if 'dview' in locals():  # locals contains list of current local variables
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(backend='local', 
                                                 n_processes=num_cpus, 
                                                 single_thread=False,
                                                 ignore_preexisting=True)
#Number of nodes in cluster 
print(f"Cluster has {n_processes} processes in the pool");

In [ ]:
cnm = load_CNMF(cnmfe_results_path, 
                n_processes, 
                dview=dview)

### Extract basic info from estimates object
Get traces, residuals, and reconstruct raw traces from cnmf object. Also, reconstruct array of frame times. 

Note we will focus on the "good" components in `cnm.estimates.idx_components` (for more on interpreting the properties in `cnm.estimates`, see https://caiman.readthedocs.io/en/master/Getting_Started.html#result-interpretation).

In [ ]:
good_inds = cnm.estimates.idx_components

denoised_traces = cnm.estimates.C[good_inds,:]  #num_components x num_frames
residuals = cnm.estimates.YrA[good_inds,:]
raw_traces = denoised_traces + residuals

num_components = denoised_traces.shape[0]
num_samples = denoised_traces.shape[1]
frame_rate = cnm.params.data['fr']
sampling_pd = 1/frame_rate
num_frames = num_samples
frame_times = np.arange(0, sampling_pd*num_samples, sampling_pd)

### Plot raw/denoised trace
Plot individual traces by hand if you want to keep things simple and use matplotlib:

In [ ]:
trace_ind = 0
raw_trace = raw_traces[trace_ind,:]
denoised_trace = denoised_traces[trace_ind,:]
residual_trace = residuals[trace_ind,:] 

plt.figure(figsize=(10,5))
plt.plot(frame_times, denoised_trace, color='k', linewidth=0.75, label='denoised')
plt.plot(frame_times, raw_trace, color='r', linewidth=0.5, label='raw')
plt.title(f"Denoised/Raw Traces (Component {trace_ind})")
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('Activity (au)');

For more fancy interactive plots, you can also use caimain's built-in viewer to view the spatial footprint and trace for any component. 

In [ ]:
template = load_template()
cnm.estimates.hv_view_components(img=template, 
                                 idx=good_inds,
                                 denoised_color='red', 
                                 cmap='gray')

## Examine the the power spectral density
Intuitively, when you inspect your calcium traces, the *signal* is dominated by lower-frequency, large-amplitude  fluctuations. The *noise* tends to consist of low-amplitude, high-frquency fluctuations. We can look at this in the power spectrum . Let's visualize this for a trace of our choosing (note that the frequency output for `welch()` is given in fraction of sampling frequency, so the maximum is `0.5`, the Nyquist frequency):

In [ ]:
# compute power spectral density
trace_ind = 0
raw_trace = raw_traces[trace_ind,:]
ps_freq, ps_dens = welch(raw_trace)

Plot the psd:

In [ ]:
plt.figure(figsize=(10,4))
plt.semilogy(ps_freq, ps_dens, marker='.', color='k');
plt.autoscale(enable=True, axis = 'x', tight = True)
plt.axvspan(0.25, 0.5, color='r', alpha=0.1)
plt.axvspan(0, 0.25, color='b', alpha=0.1)
plt.title("Power spectral density")
plt.xlabel("Frequency (fraction of sampling frequency)")
plt.ylabel("Power/Hz");

The lower-frequency region of the psd (highlighted in blue) has higher power than the lower frequencies (highlighted in red): note the y-axis is logarithmic so this is actually quite pronounced. This insight was exploited in the original CNMF paper (https://pubmed.ncbi.nlm.nih.gov/26774160/). When discussing how to estimate the noise of a trace (p 296):

> [it] can be obtained by observing the power spectral density (PSD) of [the observed calcium signal] y. The uncorrelated additive noise has flat PSD, whereas the PSD due to the calcium signal decays with the frequency as ∼(1/f2). At high frequencies, and under sparse spiking, the PSD will be dominated by the noise, and therefore an estimate σˆ2 can be obtained by averaging the PSD over a range of high frequencies. 

In practice, what we can do is get the average power in the high (red) frequency range, and take the square root to get an estimate of the standard deviation. This is what the function `GetSn()` does in caiman:

https://github.com/flatironinstitute/CaImAn/blob/79bfef180c5b2f9a0d3b65eb0618f1154d692f31/caiman/source_extraction/cnmf/deconvolution.py#L1030

If you walk through the code in `GetSn()` you might notice a few wrinkles. In particular:
- You can use the `mean`, `median`, or default `logmexp` method to get the "average" value in the red frequency range. Because the power spectrum records values across multiple orders of magnitude, the default -- that initially takes a log transform -- tends to work pretty well. 
- There is an initial division of the power by 2 in `GetSn()` because power in band-limited frequencies includes both positive and negative frequencies so is double the actual value at first (https://en.wikipedia.org/wiki/Spectral_density). 

### The upshot: creating a zscore
What we can do with our 1p data is to feed a raw trace into `GetSn()`, retrieve an estimate of our noise, and then use that as our normalization term to get a *normalized* (not just detrended) 1p calcium trace.

What is nice about this is you don't have to go through your traces and find regions of "low signal" (e.g., people often will try to eyeball their signals and say "Oh it looks like things are sort of flat here at the beginning of the trace or at this time during the trace: let's pick that as our baseline period"). Using the power spectral density removes the need to pick artibrary baseline periods from your experiments. 

The following function `zscore_trace()` calculates uses `GetSn()` to extract the noise based on the power spectral density. The trace is (optionally) shifted by some amount, and then normalized to the noise term:

In [ ]:
def zscore_trace(denoised_trace, 
                 raw_trace, 
                 offset_method = 'floor', 
                 sn_method='logmexp', 
                 range_ff=[0.25, 0.5]):
    """
    "Z-score" calcium traces based on a calculation of noise from
    power spectral density high frequency.

    Inputs:
        denoised_trace: from estimates.C
        raw_trace: from estimates.C + estimates.YrA
        offset_method: offset to use when shifting the trace (default: 'floor')
            'floor': minimum of the trace so new min will be zero
            'mean': mean of the raw/denoised trace as the zeroing point
            'median': median of raw/denoised trace
            'none': no offset just normalize
        sn_method: how are psd central values caluclated 
            mean
            median' or 'logmexp')
        range_ff: 2-elt array-like range of frequencies (input for GetSn) (default [0.25, 0.5])

    Returns 
        z_denoised: same shape as denoised_trace
        z_raw: same shape as raw trace
        trace_noise: noise level from z_raw
        
    Adapted from code by Zach Barry.
    """
    noise = GetSn(raw_trace, range_ff=range_ff, method=sn_method)  #import this from caiman

    if offset_method == 'floor':
        raw_offset = np.min(raw_trace)
        denoised_offset = np.min(denoised_trace)
    elif offset_method == 'mean':
        raw_offset = np.mean(raw_trace)
        denoised_offset = np.mean(denoised_trace)
    elif offset_method == 'median':
        raw_offset = np.median(raw_trace)
        denoised_offset = np.median(denoised_trace)
    elif offset_method == 'none':
        raw_offset = 0
        denoised_offset = 0
    else:
        raise ValueError("offset_method should be floor, mean, median, or none.")
           
    z_raw = (raw_trace - raw_offset) / noise
    z_denoised = (denoised_trace - denoised_offset)/ noise
        
    return z_denoised, z_raw, noise

print('loaded')

Applying the function to our trace above, with two different offset methods:

In [ ]:
z_denoised_floor, z_raw_floor, _ = zscore_trace(denoised_trace, 
                                                raw_trace, 
                                                offset_method='none', 
                                                sn_method='logmexp')
z_denoised_median, z_raw_median, _ = zscore_trace(denoised_trace, 
                                                  raw_trace, 
                                                  offset_method='median', 
                                                  sn_method='logmexp')

In [ ]:
f, axes = plt.subplots(2,1, figsize=(12,6))
axes[0].plot(frame_times, z_raw_floor)
axes[0].set_title('Floor', fontsize=14)
axes[0].grid()
axes[1].plot(frame_times, z_raw_median)
axes[1].set_title('Median', fontsize=14)
axes[1].grid()

Depending on the trace, you will typically find significant differences between `floor` and the other two offset methods, so you should figure out what you are most happy with. The main point, though, is that we have a calcium trace that is normalized to something meaningful: *the intrinsic noise levels in that trace*. We have something likie a traditional z score. This makes it more meaningful to compare magnitudes between components. 

Once you have settings you like you can then apply it to all traces in your data. E.g., something like: 

In [ ]:
def zscore_traces(cnm_c, cnm_yra, offset_method = 'floor',  sn_method = 'logmexp', range_ff=[0.25, 0.5]):
    """
    apply zscore_trace to all traces in estimates
    
    inputs:
        cnm_c: C array of denoised traces from cnm.estimates
        cnm_yra: YrA array of residuals from cnm.estimate
        offset_method: floor/mean/median (see zscore_trace)
        sn_method: mean/median/logmexp (see zscore_trace)
        range_ff: frequency range for GetSn
    
    outputs:
        denoised_z_traces
        raw_z_traces
        noise_all
    """
    raw_z_traces = []
    denoised_z_traces = []
    noise_all = []
    for ind, raw_trace in enumerate(cnm_yra):
        denoised_trace = cnm_c[ind,:]
        z_denoised, z_raw, noise = zscore_trace(denoised_trace,
                                                raw_trace, 
                                                offset_method=offset_method, 
                                                sn_method = sn_method,
                                                range_ff=range_ff)
        
        denoised_z_traces.append(z_denoised)
        raw_z_traces.append(z_raw)
        noise_all.append(noise)
        
    denoised_z_traces = np.array(denoised_z_traces)
    raw_z_traces = np.array(raw_z_traces)
    noise_all = np.array(noise_all)
    
    return denoised_z_traces, raw_z_traces, noise_all
print('loaded')

In [ ]:
denoised_z_traces, raw_z_traces, noise_all = zscore_traces(denoised_traces,
                                                           raw_traces, 
                                                           offset_method='floor')

Pick one and plot it

In [ ]:
trace_ind = 1
denoised_z_trace = denoised_z_traces[trace_ind,:]
plt.figure(figsize=(12,5))
plt.plot(frame_times, denoised_z_trace, color='k', linewidth=0.75)
# plt.axhspan(-3, 3, color='lime', alpha=0.2);
plt.title(f'Normalized Activity in Component {trace_ind}', fontsize=16)
plt.xlabel('Time (s)', fontsize=14)
plt.ylabel('Activity (Normalized)', fontsize=14);

And once you have your data in this form, you can then use it for all subsequent analysis.

`GetSn()`is just one of many gems hidden in the back end of caiman that is extremely useful outside of its original context.

## Comparison to standard deviation
Another common method of normalizing is to just calculate the standard deviation over the entire raw trace. You can then use that as your normalizing term. This will work, though it will tend to overestimate the noise level, because it includes the large-amplitude signal fluctuations in the calculation in addition to the noise. 

In [ ]:
std_dev_all = np.std(raw_traces, axis=1);

plt.scatter(noise_all, std_dev_all, 8, color='k', alpha=0.4);
plt.plot([0, np.max(std_dev_all)], 
         [0, np.max(std_dev_all)], 
         linestyle='--', 
         color='k')
plt.gca().set_aspect('equal')
plt.xlabel('Spectral Noise')
plt.ylabel('Standard Deviation');

It is clear that the spectral method, partly because it limits the signal's bandwidth, ends up putting consistently lower estimates on the noise. When I have played with these methods, for instance by pulling out "pure noise" stretches of 1p signals, and calculated the standard deviation on this stretch, the two methods tend to match up fairly closely (though the standard deviation is still typically higher, because the spectral technique is still bandpass-limited). I prefer the spectral technique because by filtering out the stochastic large-amplitude low-frequency fluctuations in the signals, it provides a more consistent estimate of the noise.

## Cleanup
Stop the server to save memory.

In [ ]:
cm.stop_server(dview=dview)